In [1]:
from utils import get_smiles_encodings, load_data, smile_to_hot
from sklearn.linear_model import SGDRegressor
from datetime import datetime
import numpy as np

In [2]:
file_smiles = './dataset/QM9.txt'
file_properties = './dataset/properties_QM9.npz'
smiles, alphabet, largest_molecule_len = get_smiles_encodings(file_smiles)
properties = np.load(file_properties)['properties'].astype(np.float32)
properties_txt = ['logP: represents a measure of the tendency of a compound to move from the aqueous phase into lipids',
                  'Number of rotatable bonds (RBN): the number of bonds which allow free rotation around themselves',
                  'Molecular weight (MW): the weight of a molecule based on the atomic masses of all atoms in the molecule',
                  'Number of the rings (RN): the number of connected sets of atoms and bonds in which every atom and bond is a member of a cycle']

X_train, X_test, y_train, y_test = load_data(smiles, properties)
print("X - train : ", X_train.shape)
print("y - train : ", y_train.shape)
X_hot = np.array([smile_to_hot(x, largest_molecule_len, alphabet)[1].sum(axis=0) for x in X_train])
X_hot_test = np.array([smile_to_hot(x, largest_molecule_len, alphabet)[1].sum(axis=0) for x in X_test])
print("X_hot : ", X_hot.shape)
print("X_hot_test : ", X_hot_test.shape)

X - train :  (92428,)
y - train :  (92428, 4)
X_hot :  (92428, 14)
X_hot_test :  (39612, 14)


In [3]:
model_regression_0 = SGDRegressor()
model_regression_0.fit(X_hot, y_train[:,0])
y_pred_0 = model_regression_0.predict(X_hot_test)

model_regression_1 = SGDRegressor()
model_regression_1.fit(X_hot, y_train[:,1])
y_pred_1 = model_regression_1.predict(X_hot_test)

model_regression_2 = SGDRegressor()
model_regression_2.fit(X_hot, y_train[:,2])
y_pred_2 = model_regression_2.predict(X_hot_test)

model_regression_3 = SGDRegressor()
model_regression_3.fit(X_hot, y_train[:,3])
y_pred_3 = model_regression_3.predict(X_hot_test)

In [4]:
y_test_0 = y_test[:,0]
y_test_1 = y_test[:,1]
y_test_2 = y_test[:,2]
y_test_3 = y_test[:,3]    

In [5]:
sum_0 = 0
for i in range(y_test_0.shape[0]):
    sum_0 = sum_0 + np.abs(np.abs(y_pred_0[i]) - np.abs(y_test_0[i]))
sum_0 = sum_0 / y_test_0.shape[0]

print(properties_txt[0])
print()
print("Number of unique values in y_test[:,0] : ", np.unique(y_test_0).shape[0])
print("Min of y_test[:,0] : ", np.min(y_test_0))
print("Max of y_test[:,0] : ", np.max(y_test_0))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_0).shape[0])
print("Min of y_pred : ", np.min(y_pred_0))
print("Max of y_pred : ", np.max(y_pred_0))
print()
print("Error is on avearage : ", sum_0)
print("Let's say the target value is : 1.0, then the model would guess a value in the following interval :")
print(" -> [", 1.0 - sum_0,",",1.0 + sum_0,"]")

logP: represents a measure of the tendency of a compound to move from the aqueous phase into lipids

Number of unique values in y_test[:,0] :  6400
Min of y_test[:,0] :  -2.8142
Max of y_test[:,0] :  3.7569

Number of unique values in y_pred :  1337
Min of y_pred :  -2.9340682986444517
Max of y_pred :  3.0984655757689286

Error is on avearage :  0.34742620249141765
Let's say the target value is : 1.0, then the model would guess a value in the following interval :
 -> [ 0.6525737975085824 , 1.3474262024914176 ]


In [6]:
sum_1 = 0
sum_1_5 = 0
for i in range(y_test_1.shape[0]):
    sum_1 = sum_1 + np.abs(np.abs(y_pred_1[i]) - np.abs(y_test_1[i]))
    if(y_pred_1[i] == y_test_1[i]):
        sum_1_5 = sum_1_5 + 1
sum_1 = sum_1 / y_test_1.shape[0]
sum_1_5_r = sum_1_5 / y_test_1.shape[0] * 100

print(properties_txt[1])
print()
print("Number of unique values in y_test[:,1] : ", np.unique(y_test_1).shape[0])
print("Min of y_test[:,1] : ", np.min(y_test_1))
print("Max of y_test[:,1] : ", np.max(y_test_1))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_1).shape[0])
print("Min of y_pred : ", np.min(y_pred_1))
print("Max of y_pred : ", np.max(y_pred_1))
print()
print("Error is on avearage : ", sum_1)
print("There were ", sum_1_5, " correct predictions, out of ", y_test_1.shape[0], ", which is ", 100 - sum_1_5_r,"% error rate")
print()
print("Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : ")
print()

y_pred_1_r = np.around(y_pred_1)

sum_1_r = 0
sum_1_5_r = 0
for i in range(y_pred_1_r.shape[0]):
    sum_1_r = sum_1_r + np.abs(np.abs(y_pred_1_r[i]) - np.abs(y_test_3[i]))
    if(y_pred_1_r[i] == y_test_3[i]):
        sum_1_5_r = sum_1_5_r + 1
sum_1_r = sum_1_r / y_test_3.shape[0]
sum_1_5_r_ = sum_1_5_r / y_test_3.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_1_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_1_r))
print("Max of y_pred_rounded : ", np.max(y_pred_1_r))
print()
print("Error is on avearage : ", sum_1_r)
print("There were ", sum_1_5_r, " correct predictions, out of ", y_test_3.shape[0], ", which is ", 100 - sum_1_5_r_,"% error rate")

Number of rotatable bonds (RBN): the number of bonds which allow free rotation around themselves

Number of unique values in y_test[:,1] :  7
Min of y_test[:,1] :  0.0
Max of y_test[:,1] :  6.0

Number of unique values in y_pred :  1337
Min of y_pred :  -0.9429359085231339
Max of y_pred :  3.0945099943311227

Error is on avearage :  0.7241496599004495
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : 

Number of unique values in y_pred_rounded :  5
Min of y_pred_rounded :  -1.0
Max of y_pred_rounded :  3.0

Error is on avearage :  1.1161516712107442
There were  11443  correct predictions, out of  39612 , which is  71.11228920529132 % error rate


In [7]:
sum_2 = 0
sum_2_r = 0
for i in range(y_test_2.shape[0]):
    sum_2 = sum_2 + np.abs(np.abs(y_pred_2[i]) - np.abs(y_test_2[i]))
    sum_2_r = sum_2_r + np.abs(100 - (np.abs(y_pred_2[i]) * 100 / np.abs(y_test_2[i])))
sum_2 = sum_2 / y_test_2.shape[0]
sum_2_r = sum_2_r / y_test_2.shape[0]

print(properties_txt[2])
print()
print("Number of unique values in y_test[:,2] : ", np.unique(y_test_2).shape[0])
print("Min of y_test[:,2] : ", np.min(y_test_2))
print("Max of y_test[:,2] : ", np.max(y_test_2))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_2).shape[0])
print("Min of y_pred : ", np.min(y_pred_2))
print("Max of y_pred : ", np.max(y_pred_2))
print()
print("Error is on avearage : ", sum_2)
print("Error rate is on avearage : ", sum_2_r, "% -> for each guess, the model is of by ", sum_2_r, "%")
print("Let's say the target value is : 150, then the model would guess a value in the following interval :")
print(" -> [", 150 - sum_2,",",150 + sum_2,"]")

Molecular weight (MW): the weight of a molecule based on the atomic masses of all atoms in the molecule

Number of unique values in y_test[:,2] :  222
Min of y_test[:,2] :  104.152
Max of y_test[:,2] :  152.037

Number of unique values in y_pred :  1337
Min of y_pred :  104.17213730410894
Max of y_pred :  150.82148444008834

Error is on avearage :  0.02814579452637598
Error rate is on avearage :  0.021638029627023953 % -> for each guess, the model is of by  0.021638029627023953 %
Let's say the target value is : 150, then the model would guess a value in the following interval :
 -> [ 149.97185420547362 , 150.02814579452638 ]


In [8]:
sum_3 = 0
sum_3_5 = 0
for i in range(y_test_3.shape[0]):
    sum_3 = sum_3 + np.abs(np.abs(y_pred_3[i]) - np.abs(y_test_3[i]))
    if(y_pred_3[i] == y_test_3[i]):
        sum_3_5 = sum_3_5 + 1
sum_3 = sum_3 / y_test_1.shape[0]
sum_3_5_r = sum_3_5 / y_test_1.shape[0] * 100

print(properties_txt[3])
print()
print("Number of unique values in y_test[:,3] : ", np.unique(y_test_3).shape[0])
print("Min of y_test[:,3] : ", np.min(y_test_3))
print("Max of y_test[:,3] : ", np.max(y_test_3))
print()
print("Number of unique values in y_pred : ", np.unique(y_pred_3).shape[0])
print("Min of y_pred : ", np.min(y_pred_3))
print("Max of y_pred : ", np.max(y_pred_3))
print()
print("Error is on avearage : ", sum_3)
print("There were ", sum_3_5, " correct predictions, out of ", y_test_3.shape[0], ", which is ", 100 - sum_3_5_r,"% error rate")
print()
print("Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : ")
print()

y_pred_3_r = np.around(y_pred_3)

sum_3_r = 0
sum_3_5_r = 0
for i in range(y_test_3.shape[0]):
    sum_3_r = sum_3_r + np.abs(np.abs(y_pred_3_r[i]) - np.abs(y_test_3[i]))
    if(y_pred_3_r[i] == y_test_3[i]):
        sum_3_5_r = sum_3_5_r + 1
sum_3_r = sum_3_r / y_test_3.shape[0]
sum_3_5_r_ = sum_3_5_r / y_test_3.shape[0] * 100

print("Number of unique values in y_pred_rounded : ", np.unique(y_pred_3_r).shape[0])
print("Min of y_pred_rounded : ", np.min(y_pred_3_r))
print("Max of y_pred_rounded : ", np.max(y_pred_3_r))
print()
print("Error is on avearage : ", sum_3_r)
print("There were ", sum_3_5_r, " correct predictions, out of ", y_test_3.shape[0], ", which is ", 100 - sum_3_5_r_,"% error rate")

Number of the rings (RN): the number of connected sets of atoms and bonds in which every atom and bond is a member of a cycle

Number of unique values in y_test[:,3] :  9
Min of y_test[:,3] :  0.0
Max of y_test[:,3] :  8.0

Number of unique values in y_pred :  1337
Min of y_pred :  -0.207673763526039
Max of y_pred :  6.0036415792801066

Error is on avearage :  0.13921218045050776
There were  0  correct predictions, out of  39612 , which is  100.0 % error rate

Since the number of unique values is small, let's round the y_pred, to see if we get a better approximation : 

Number of unique values in y_pred_rounded :  7
Min of y_pred_rounded :  -0.0
Max of y_pred_rounded :  6.0

Error is on avearage :  0.06652024638998283
There were  37399  correct predictions, out of  39612 , which is  5.586690901746948 % error rate
